In [32]:
# start coding here
mmseqs2File = snakemake.input.mmseqsPdbIDs
scopFile = snakemake.input.scopPdbIDs[0]

outFile = snakemake.output[0]

In [50]:
from collections import defaultdict
import json

def _convert_mmseqs2_to_dict(mmseqs2File):
    mmseqs2Clusters = defaultdict(list)
    with open(mmseqs2File, "r") as f:
        for line in f.readlines():
            clusterID, clusterMember = line.split()
            mmseqs2Clusters[clusterID].append(clusterMember)
    return mmseqs2Clusters

def _read_scopFile(scopFile):
    with open(scopFile, "r") as f:
        return json.load(f)

def main(mmseqs2File, scopFile):
    mmseqs2Clusters = _convert_mmseqs2_to_dict(mmseqs2File)
    
    scopPdbIDs = _read_scopFile(scopFile)
    
    scopClusters = list() # track clusterIDs that include scopIDs
    for i, (clusterID, clusterMembers) in enumerate(mmseqs2Clusters.items()):
        print(f"Checking Cluster {i+1} of {len(mmseqs2Clusters)} for scop members", end="\r")
        # check if there is a shared member between each list
        if not set(clusterMembers).isdisjoint(scopPdbIDs):
            scopClusters.append(clusterID)
    
    return scopClusters

In [55]:
%%timeit -r 1 -n 1
import json

scopClusters = main(mmseqs2File, scopFile)

with open(outFile, "w") as f:
    json.dump(scopClusters, f)